# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingenería en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Examen 01**:

**Fecha**: 14/03/2024

**Nombre del Estudiante**: Alejandro Samuel Romero Mora

**Profesor**: Pablo Camarillo Ramirez

In [2]:
import findspark
findspark.init()

In [38]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-Alejandro-Samuel-Romero-Mora") \
    .master("spark://b7351070baae:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

25/03/14 14:39:29 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Hacemos los esquemas de cada uno

In [39]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

employees_schema = StructType([
    StructField("employee_id", IntegerType(), True),
    StructField("employee_info", StringType(), True)
])
departments_schema = StructType([
    StructField("department_id", IntegerType(), True),
    StructField("department_name", StringType(), True),
    StructField("location", StringType(), True)
])


Hacemos los df

In [40]:
employees_df = spark.read \
    .schema(employees_schema) \
    .option("header", "true") \
    .csv("/home/jovyan/notebooks/data/employees.csv")

departments_df = spark.read \
    .schema(departments_schema) \
    .option("header", "true") \
    .csv("/home/jovyan/notebooks/data/departments.csv")

Imprimimos 

In [41]:
print("Esquema de empleados:")
employees_df.printSchema()
print("Datos de empleados:")
employees_df.show(5, truncate=False)

Esquema de empleados:
root
 |-- employee_id: integer (nullable = true)
 |-- employee_info: string (nullable = true)

Datos de empleados:
+-----------+-------------------------------------------------------------------------------------------+
|employee_id|employee_info                                                                              |
+-----------+-------------------------------------------------------------------------------------------+
|1          |{'name': 'Caitlyn', 'department_id': 103, 'salary': 115959.78, 'hire_date': '2002-06-10'}  |
|2          |{'name': 'Rachel', 'department_id': 104, 'salary': 100820.16, 'hire_date': '2009-07-01'}   |
|3          |{'name': 'Carrie', 'department_id': 105, 'salary': 114421.44, 'hire_date': '1998-12-10'}   |
|4          |{'name': 'Renee', 'department_id': 104, 'salary': 54688.13, 'hire_date': '1995-03-17'}     |
|5          |{'name': 'Gabriella', 'department_id': 109, 'salary': 106267.03, 'hire_date': '1995-02-09'}|
+-----------+--

In [42]:
print("Esquema de departamentos:")
departments_df.printSchema()
print("Datos de departamentos:")
departments_df.show(5, truncate=False)

Esquema de departamentos:
root
 |-- department_id: integer (nullable = true)
 |-- department_name: string (nullable = true)
 |-- location: string (nullable = true)

Datos de departamentos:
+-------------+----------------------+---------+
|department_id|department_name       |location |
+-------------+----------------------+---------+
|101          |Human Resources       |San Diego|
|102          |Finance and Accounting|New York |
|103          |Sales and Marketing   |Chicago  |
|104          |Data Engineering      |Zapopan  |
|105          |Data Science          |Seattle  |
+-------------+----------------------+---------+
only showing top 5 rows



In [43]:
from pyspark.sql.functions import get_json_object, to_date

employees_df = employees_df.withColumn("name", get_json_object(employees_df.employee_info, "$.name")) \
                           .withColumn("department_id", get_json_object(employees_df.employee_info, "$.department_id").cast("int")) \
                           .withColumn("salary", get_json_object(employees_df.employee_info, "$.salary").cast("double")) \
                           .withColumn("hire_date", to_date(get_json_object(employees_df.employee_info, "$.hire_date")))

print("Esquema de Employees después:")
employees_df.printSchema()
print("Datos de employees después:")
employees_df.show(5, truncate=False)

Esquema de Employees después:
root
 |-- employee_id: integer (nullable = true)
 |-- employee_info: string (nullable = true)
 |-- name: string (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)

Datos de employees después:
+-----------+-------------------------------------------------------------------------------------------+---------+-------------+---------+----------+
|employee_id|employee_info                                                                              |name     |department_id|salary   |hire_date |
+-----------+-------------------------------------------------------------------------------------------+---------+-------------+---------+----------+
|1          |{'name': 'Caitlyn', 'department_id': 103, 'salary': 115959.78, 'hire_date': '2002-06-10'}  |Caitlyn  |103          |115959.78|2002-06-10|
|2          |{'name': 'Rachel', 'department_id': 104, 'salary': 100820.16, 'hire_dat

Hacemos el Join e imprimimos para ver si lo hicimos correctamente

In [44]:
new_employees_df = employees_df.join(
    departments_df,
    on="department_id",
    how="left"
)

print("Esquema de employees nuevo:")
new_employees_df.printSchema()
print("Nuevos datos:")
new_employees_df.show(5, truncate=False)


Esquema de employees nuevo:
root
 |-- department_id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- employee_info: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- department_name: string (nullable = true)
 |-- location: string (nullable = true)

Nuevos datos:
+-------------+-----------+-------------------------------------------------------------------------------------------+---------+---------+----------+-------------------+-------------+
|department_id|employee_id|employee_info                                                                              |name     |salary   |hire_date |department_name    |location     |
+-------------+-----------+-------------------------------------------------------------------------------------------+---------+---------+----------+-------------------+-------------+
|103          |1          |{'name': 'Caitlyn', 'department_id'

Agregamos una nuevo columna y le añadimos la condicional para guardarlo en High o Low

In [45]:
from pyspark.sql.functions import when

new_employees_df = new_employees_df.withColumn(
    "salary_category",
    when(new_employees_df.salary >= 55000, "High").otherwise("Low")
)

Verificamos si se hizo correctamente la nueva columna

In [46]:
print("Esquema de new_employees_df con salary_category:")
new_employees_df.printSchema()
print("Datos de new_employees_df con salary_category:")
new_employees_df.show(5, truncate=False)

Esquema de new_employees_df con salary_category:
root
 |-- department_id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- employee_info: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- department_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- salary_category: string (nullable = false)

Datos de new_employees_df con salary_category:
+-------------+-----------+-------------------------------------------------------------------------------------------+---------+---------+----------+-------------------+-------------+---------------+
|department_id|employee_id|employee_info                                                                              |name     |salary   |hire_date |department_name    |location     |salary_category|
+-------------+-----------+--------------------------------------------------------------------------------------

Hacemos los dataframes filtrados

In [47]:
high_salary_df = new_employees_df.filter(new_employees_df.salary_category == "High")
low_salary_df = new_employees_df.filter(new_employees_df.salary_category == "Low")

print("Empleados con salario 'High':")
high_salary_df.show(5, truncate=False)

print("Empleados con salario 'Low':")
low_salary_df.show(5, truncate=False)

Empleados con salario 'High':
+-------------+-----------+-------------------------------------------------------------------------------------------+---------+---------+----------+-------------------+-------------+---------------+
|department_id|employee_id|employee_info                                                                              |name     |salary   |hire_date |department_name    |location     |salary_category|
+-------------+-----------+-------------------------------------------------------------------------------------------+---------+---------+----------+-------------------+-------------+---------------+
|103          |1          |{'name': 'Caitlyn', 'department_id': 103, 'salary': 115959.78, 'hire_date': '2002-06-10'}  |Caitlyn  |115959.78|2002-06-10|Sales and Marketing|Chicago      |High           |
|104          |2          |{'name': 'Rachel', 'department_id': 104, 'salary': 100820.16, 'hire_date': '2009-07-01'}   |Rachel   |100820.16|2009-07-01|Data Engineering

Ahora calculamos el promedio de cada uno con la función de average

In [48]:
from pyspark.sql.functions import avg
high_salary_avg_df = high_salary_df.groupBy("department_name") \
                                   .agg(avg("salary").alias("avg_salary")) \
                                   .select("department_name", "avg_salary")
low_salary_avg_df = low_salary_df.groupBy("department_name") \
                                 .agg(avg("salary").alias("avg_salary")) \
                                 .select("department_name", "avg_salary")
print("Salario promedio por departamento (High):")
high_salary_avg_df.show(truncate=False)

print("Salario promedio por departamento (Low):")
low_salary_avg_df.show(truncate=False)

Salario promedio por departamento (High):
+-------------------------------------------+------------------+
|department_name                            |avg_salary        |
+-------------------------------------------+------------------+
|Corporate Strategy and Business Development|102741.38324414717|
|Sales and Marketing                        |100839.65275449108|
|Data Engineering                           |101626.29492163012|
|Research and Development                   |98714.3003086419  |
|Finance and Accounting                     |100731.07877887784|
|Customer Service                           |101585.01600000002|
|Legal                                      |99366.3129102167  |
|Data Science                               |101903.63710344829|
|Operations                                 |100169.65621722837|
|Human Resources                            |104999.43191489363|
+-------------------------------------------+------------------+

Salario promedio por departamento (Low):
+-----

Ahora obtenemos los top

In [49]:
from pyspark.sql.functions import col

top_5_high_salary_df = new_employees_df.filter(new_employees_df.salary_category == "High") \
                                       .orderBy(col("salary").desc()) \
                                       .limit(5)

top_5_low_salary_df = new_employees_df.filter(new_employees_df.salary_category == "Low") \
                                      .orderBy(col("salary").desc()) \
                                      .limit(5)

print("Top 5 empleados con salario 'High':")
top_5_high_salary_df.show(truncate=False)

print("Top 5 empleados con salario 'Low':")
top_5_low_salary_df.show(truncate=False)

Top 5 empleados con salario 'High':
+-------------+-----------+-------------------------------------------------------------------------------------------+---------+---------+----------+-------------------------------------------+-----------+---------------+
|department_id|employee_id|employee_info                                                                              |name     |salary   |hire_date |department_name                            |location   |salary_category|
+-------------+-----------+-------------------------------------------------------------------------------------------+---------+---------+----------+-------------------------------------------+-----------+---------------+
|101          |1778       |{'name': 'Gabriella', 'department_id': 101, 'salary': 149989.73, 'hire_date': '2018-09-14'}|Gabriella|149989.73|2018-09-14|Human Resources                            |San Diego  |High           |
|101          |3621       |{'name': 'Katherine', 'department_id': 101, '

In [50]:
from pyspark.sql.functions import datediff, current_date

new_employees_df = new_employees_df.withColumn(
    "years_in_company",
    (datediff(current_date(), col("hire_date")) / 365).cast("int")
)
print("Esquema de new_employees_df con years_in_company:")
new_employees_df.printSchema()
print("Datos de new_employees_df con years_in_company:")
new_employees_df.show(5, truncate=False)


Esquema de new_employees_df con years_in_company:
root
 |-- department_id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- employee_info: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- department_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- salary_category: string (nullable = false)
 |-- years_in_company: integer (nullable = true)

Datos de new_employees_df con years_in_company:
+-------------+-----------+-------------------------------------------------------------------------------------------+---------+---------+----------+-------------------+-------------+---------------+----------------+
|department_id|employee_id|employee_info                                                                              |name     |salary   |hire_date |department_name    |location     |salary_category|years_in_company|
+-------------+-----------+-

In [4]:
sc.stop()
 